### House Sales in Metropolitan Melbourne in 2016 and 2017
The sales data are stored in **.csv** files that contain the following columns:
#### ID
Row record ID.
#### Suburb
The suburb where the property is located.
#### Address
The property address.
#### Postcode
The property postcode, a 4-digit number.
#### CouncilArea
The municipality or council area under which the property falls.
#### Regionname
The region of metropolitan Melbourne under which the property falls.
#### Price
How much the property sold for. This value is a whole number.
#### SaleDate
The date of property sale, in the dd/mm/yyyy format.
#### Seller
The real estate agency that sold the property.
#### Bedrooms
Number of bedrooms.
#### Bathrooms
Number of bathrooms.
#### CarSpaces
Number of car spaces.
#### Landsize
Size of the land. This value is a whole number.
#### Latitude
The latitude geolocation coordinate of the property
#### Longitude
The longitude geolocation coordinate of the property

In [ ]:
import csv


def read_data(filename):

    data = {}
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            ID = row["ID"]
            del row["ID"]
            data[ID] = dict(row)
            
    return data


data_noisy = read_data()

Create a shadow copy of dictionary input which functions as deep copy.

In [ ]:
def manual_deep_copy(data_original):
    
    data_copy = {}
    for house_id, house_info in data_original.items():
        # Different dictionaries pointing to same strings that are immutable
        data_copy[house_id] = house_info.copy() 
    
    return data_copy

Valid values in the CouncilArea and Regionname are enumerated.

In [ ]:
VALID_REGION_NAMES = [
    "Eastern Metropolitan", "Northern Metropolitan", "South-Eastern Metropolitan", 
    "Southern Metropolitan", "Western Metropolitan"
]

VALID_COUNCIL_AREAS = [
    "Banyule", "Brimbank", "Darebin", "Hume", "Knox", "Maribyrnong", 
    "Melbourne", "Moonee Valley", "Moreland", "Whittlesea", "Yarra"
]

Values in the CouncilArea column could be incorrectly spelt or contain extra characters. Examples of such incorrect values are: Morelnd; Yara; nox; aDarebin; Melbun; Honey Valley.

Incorrect values are updated to correct council area values with the highest set similarities:
$$
\text{Sim}(\text{string1}, \text{string2}) = \frac{|S_1 \cap S_2|}{|S_1 \cup S_2|}
$$
In cases where an incorrect string has the same similarity score with two or more correct council area values, it just replaces the incorrect value with None. 

In [ ]:
def set_similarity(s1, s2):
    
    set_1 = set(s1.lower())
    set_2 = set(s2.lower())
    
    return len(set_1 & set_2) / len(set_1 | set_2)

Some people have formatted the sale date incorrectly, such that it is either not of the form dd/mm/yyyy (variations d/mm/yyyy, dd/m/yyyy and d/m/yyyy would also be valid) or contains invalid dates, such as 31/11/2016.

If a date does not fall under the year 2016 or the year 2017, it is also considered invalid.

In [ ]:
def check_date(date_input):
    
    date_l = date_input.split('/')
    if len(date_l) != 3:  # If date not in 'day/month/year' format
        return None
    else:
        d, m, y = list(map(int, date_l))
        if y not in [2016, 2017]:
            return None
        else:
            if m in [1, 3, 5, 7, 8, 10, 12]:
                d_max = 31
            elif m in [4, 6, 9, 11]:
                d_max = 30
            elif m == 2:
                if y == 2016:
                    d_max = 29
                else:
                    d_max = 28
            else:  # If m not between 1 and 12
                return None
            if d < 1 or d > d_max:
                return None
            return date_input

Other potential errors:

- Some values in Price column are not numeric whole numbers.

- Some Regionname values are no longer current and valid region.

- Some Postcode values are not valid Melbourne postcode values: they need to be 4-digit integers that start with 3.

Return a **new** dictionary which is identical to the input dictionary, except that invalid data, as described by the rules above, have been replaced with None, and CouncilArea is updated:

In [ ]:
def sales_data_clean(data):
    
    ## Clean the sales data and replace the incorrect CouncilArea
    
    data_clean = manual_deep_copy(data)
    
    for house_info in data_clean.values():        
        if not house_info['Price'].isdigit():
            house_info['Price'] = None
        if not house_info['Regionname'] in VALID_REGION_NAMES:
            house_info['Regionname'] = None
        
        house_info['SaleDate'] = check_date(house_info['SaleDate'])
        
        pc = house_info['Postcode']
        if not pc.isdigit() or len(pc) != 4 or pc[0] != '3':
            house_info['Postcode'] = None
        
        if house_info['CouncilArea'] not in VALID_COUNCIL_AREAS:
            sim_l = []
            for council in VALID_COUNCIL_AREAS:
                sim = set_similarity(house_info['CouncilArea'], council)
                sim_l.append(sim)
            if sim_l.count(max(sim_l)) > 1:
                house_info['CouncilArea'] = None
            else:
                max_index = sim_l.index(max(sim_l))
                house_info['CouncilArea'] = VALID_COUNCIL_AREAS[max_index]
    
    return data_clean

A function called seller_sale_stats(data, seller, year) which takes three arguments: a dictionary of clean data, a name of real estate agency, and a year.

For the given agency and the given year, calculate the following sales stats:

- median sale Price, rounded to the whole number. If there is an even number of numbers, then the median is the average of the two middle numbers. 

- sum total Price.

- minimum instance of Price, presented as a tuple consisting of the price and the suburb with the minimum price. If more than one suburb has the minimum price, return whichever appears last in the CSV file.

- maximum instance of Price, presented as a tuple consisting of the price and the suburb with the maximum price. If more than one suburb has the maximum price, return whichever appears last in the CSV file.

If a seller does not have any sales information for the year, then the values are set to 0.

In [ ]:
def seller_sale_stats(data, seller, year):
    
    ## Get a summary of sales by a particular seller in a particular year
    
    sales = []
    stats = {'max': 0, 'med': 0, 'min': 0, 'sum': 0}
    
    for house_info in data.values():
        if (
            house_info['Seller'] == seller
            and house_info['SaleDate'][-4:] == year
        ):
            sales.append((int(house_info['Price']), house_info['Suburb']))
    n = len(sales)
    if n > 0:
        
        # Keep the CSV order of suburbs stable, use iteration rather than sort
        # Get the last tuple containing the min and max prices
        price_min = min(sales)[0]
        price_max = max(sales)[0]
        for i in range(n):
            if sales[i][0] == price_min:
                stats['min'] = sales[i]
            if sales[i][0] == price_max:
                stats['max'] = sales[i]
                
        # Get the median price
        sales_sorted = sorted(sales)
        if n % 2 == 0:
            stats['med'] = round(
                (sales_sorted[n // 2 - 1][0] + sales_sorted[n // 2][0]) / 2
            )
        else:
            stats['med'] = sales_sorted[n // 2][0]
            
        # Find the summation of sales prices
        stats['sum'] = sum(sales[j][0] for j in range(n))

    return {'seller': seller, 'stats': stats}

The similarity distance between two properties is defined by the fields of Bedrooms, Bathrooms, CarSpaces and Landsize. Start with a similarity_distance score of 0, and proceed as follows:

- add the difference in number of bedrooms between the two properties to the similarity_distance score
- add the difference in number of bathrooms between the two properties to the similarity_distance score
- add the difference in number of car spaces between the two properties to the similarity_distance score
- select the smaller of the two land sizes and divide that number into the larger landsize. Add the result (rounded to 2 decimal place) to the similarity_distance score

The lower a similarity_distance score, the more similar two properties are (if two properties have exactly the same values for all of these four fields, then similarity_distance = 1).

In [ ]:
def sim_distance(house_info1, house_info2):
    
    ## Calculate similarity distance score of two properties
    
    dif = sum(
        abs(int(house_info1[info]) - int(house_info2[info])) 
        for info in ['Bedrooms', 'Bathrooms', 'CarSpaces']
    )
    dif += (
        max(int(house_info1['Landsize']), int(house_info2['Landsize'])) / 
        min(int(house_info1['Landsize']), int(house_info2['Landsize']))
    )
    
    return round(dif, 2)


A function price_comparison(sale_id, data, suburb), where:

- sale_id is the ID of a sale record to be compared against
- data is a dictionary of clean data
- suburb is the name of a suburb from which we are finding comparison sales

The price_comparison() function is to retrieve the property in suburb that is most similar to the property associated with sale_id, and get the sale price of that property.

- If there is more than one such most similar property, obtain the average sale price of them (rounded to the closest integer). Once this sale price is determined, the function will return a string message in the form of the examples below.

- If the given input sale_id does does correspond to a sales record, then return the following string message: "There is no sales record with the input ID provided".

- If the input comparison suburb has no sales records to compare against, then return the following string message: "There are no sales records in the comparison suburb provided".

Unless otherwise specified, when dealing with float results throughout your code, round to 2 decimal places.

In [ ]:
def price_comparison(input_sale_id, data, suburb):
    
    ## Find the most similar houses in the suburb and report prices to compare
    
    if not input_sale_id in data:
        return 'There is no sales record with the input ID provided'
    
    # Traverse data, only keep properties with the minimum distance
    min_distance = float('inf')
    for house_id, house_info in data.items():
        if house_info['Suburb'] == suburb and house_id != input_sale_id:
            distance = sim_distance(house_info, data[input_sale_id])
            if distance < min_distance:
                min_distance = distance
                similar_properties = [house_id]
            elif distance == min_distance:
                similar_properties.append(house_id)
            
    if min_distance == float('inf'):  # similar_properties not initialized
        return 'There are no sales records in the comparison suburb provided'
    
    # Calculate the average price for similar properties
    price_sum = sum(
        int(data[id_sim]['Price']) for id_sim in similar_properties
    )
    price_avg = round(price_sum / len(similar_properties))
    
    string_about_input = (
        f"The input property at {data[input_sale_id]['Address']} "
        f"in the suburb of {data[input_sale_id]['Suburb']} has a "
        f"sale price of {data[input_sale_id]['Price']}\n"
    )
    if len(similar_properties) == 1:
        string_about_similar = (
            f"The most similar property in the suburb "
            f"of {suburb} sold for {price_avg}"
        )
    else:
        string_about_similar = (
            f"The most similar properties in the suburb "
            f"of {suburb} sold for an average price of {price_avg}"
        )
    return string_about_input + string_about_similar

JSON structure inputs contain core address information (Suburb, Address and Postcode) for a set of properties and looks like the following example:

[
    {"Suburb": "Rosanna", "Address": "7 Hylton Cr", "Postcode": 3084},
    {"Suburb": "Preston", "Address": "3/152 Tyler St", "Postcode": 3072},
    {"Suburb": "Epping", "Address": "19 Houston St", "Postcode": 3076}
]

The function provide_realestate_info will return a JSON that adds more information to the input structure. For each property in this input set, the function searches the main dataset for a record of this property. 

If the record for a property is found, then add the key/value elements for the fields Price, Bedrooms and Landsize. If the input property record is not found, then remove it from the set.

In [ ]:
import json


def add_info(query_entry, house_info):
    
    ## Update address, add price, bedrooms, & land size for one queried property
    
    query_entry['Address'] = house_info['Address']  # Ensure complete address 
    query_entry['Bedrooms'] = int(house_info['Bedrooms'])
    query_entry['Landsize'] = int(house_info['Landsize'])
    query_entry['Price'] = int(house_info['Price'])
    return query_entry
    

def provide_realestate_info(main_data, queried_properties):
    
    ## Add price, bedrooms and land size info to the queried properties
    
    to_query = json.loads(queried_properties)
    
    output = []  # Output should be a list of JSON objects
    for entry in to_query:  # Each entry is a dictionary of a property location
        candidates = []
        for house_info in main_data.values():
            if (
                house_info['Suburb'] == entry['Suburb'] 
                and int(house_info['Postcode']) == entry['Postcode']
                and entry['Address'] in house_info['Address']
            ):
                candidates.append(house_info)
        if len(candidates) == 1:  # Entry can uniquely identify one property
            add_info(entry, candidates[0])
            output.append(entry)
    
    return output